In [1]:
import json
import orjson

orjson.__version__

'3.7.0'

# 2 orjson常用方法

## 2.1 序列化

In [2]:
import random

demo_json1 = [
    {
        'id': 999999,
        'value': random.uniform(0, 1000)
    }
    for i in range(10000000)
]

In [3]:
%time _ = json.dumps(demo_json1)

CPU times: total: 16.3 s
Wall time: 18 s


In [4]:
%time _ = orjson.dumps(demo_json1)

CPU times: total: 1.33 s
Wall time: 1.32 s


## 2.2 反序列化

In [5]:
_ = json.dumps(demo_json1)

In [6]:
%time loads_result = json.loads(_)

CPU times: total: 9.95 s
Wall time: 10.5 s


In [7]:
_ = orjson.dumps(demo_json1)

In [8]:
%time loads_result = orjson.loads(_)

CPU times: total: 3.72 s
Wall time: 3.73 s


## 2.3 丰富的option选项

- OPT_INDENT_2

In [9]:
demo_json2 = {"a": "b", "c": {"d": True}, "e": [1, 2]}
print(orjson.dumps(demo_json2).decode())

{"a":"b","c":{"d":true},"e":[1,2]}


In [10]:
print(orjson.dumps(demo_json2, option=orjson.OPT_INDENT_2).decode())

{
  "a": "b",
  "c": {
    "d": true
  },
  "e": [
    1,
    2
  ]
}


- OPT_OMIT_MICROSECONDS

In [11]:
from datetime import datetime

orjson.dumps({'now': datetime.now()})

b'{"now":"2022-06-05T19:25:13.497548"}'

In [12]:
orjson.dumps({'now': datetime.now()},
             option=orjson.OPT_OMIT_MICROSECONDS)

b'{"now":"2022-06-05T19:25:13"}'

- OPT_NON_STR_KEYS

In [13]:
orjson.dumps({1: 'a', 2: 'b'})

TypeError: Dict key must be str

In [14]:
orjson.dumps({1: 'a', 2: 'b'}, option=orjson.OPT_NON_STR_KEYS)

b'{"1":"a","2":"b"}'

- OPT_SERIALIZE_NUMPY

In [15]:
import numpy as np

demo_json3 = {
    'numpy-demo': np.random.randint(0, 10, (10, 10))
}
demo_json3

{'numpy-demo': array([[2, 0, 0, 0, 9, 5, 8, 5, 6, 3],
        [0, 5, 3, 8, 6, 7, 9, 1, 6, 4],
        [5, 1, 4, 8, 3, 3, 0, 7, 1, 2],
        [5, 0, 7, 0, 8, 1, 8, 9, 4, 5],
        [5, 2, 0, 1, 7, 6, 6, 4, 5, 0],
        [9, 3, 6, 6, 1, 6, 0, 1, 0, 8],
        [6, 7, 6, 3, 9, 9, 8, 3, 3, 8],
        [1, 6, 0, 7, 9, 1, 1, 7, 4, 4],
        [4, 8, 7, 6, 0, 0, 9, 6, 1, 5],
        [9, 2, 8, 0, 7, 4, 5, 6, 7, 2]])}

In [16]:
orjson.dumps(demo_json3, option=orjson.OPT_SERIALIZE_NUMPY)

b'{"numpy-demo":[[2,0,0,0,9,5,8,5,6,3],[0,5,3,8,6,7,9,1,6,4],[5,1,4,8,3,3,0,7,1,2],[5,0,7,0,8,1,8,9,4,5],[5,2,0,1,7,6,6,4,5,0],[9,3,6,6,1,6,0,1,0,8],[6,7,6,3,9,9,8,3,3,8],[1,6,0,7,9,1,1,7,4,4],[4,8,7,6,0,0,9,6,1,5],[9,2,8,0,7,4,5,6,7,2]]}'

- OPT_SERIALIZE_UUID

In [17]:
import uuid

demo_json4 = {
    'uuid_demo': uuid.uuid4()
}
demo_json4

{'uuid_demo': UUID('72505cf9-dd02-4ce7-ae9a-1715627816b5')}

In [18]:
orjson.dumps(demo_json4)

b'{"uuid_demo":"72505cf9-dd02-4ce7-ae9a-1715627816b5"}'

- OPT_SORT_KEYS

In [19]:
orjson.dumps({"b": 1, "c": 2, "a": 3})

b'{"b":1,"c":2,"a":3}'

In [20]:
orjson.dumps({"b": 1, "c": 2, "a": 3},
             option=orjson.OPT_SORT_KEYS)

b'{"a":3,"b":1,"c":2}'

- 组合多种option

In [21]:
demo_json5 = {
    'c': np.random.randint(0, 10, (2, 2)),
    'd': np.random.randint(0, 10, (2, 2)),
    'a': np.random.randint(0, 10, (2, 2))
}

orjson.dumps(demo_json5,
             option=orjson.OPT_SERIALIZE_NUMPY | orjson.OPT_SORT_KEYS)

b'{"a":[[4,0],[3,5]],"c":[[7,6],[5,9]],"d":[[7,7],[8,8]]}'

## 2.4 针对dataclass、datetime、subclass添加自定义处理策略

In [22]:
from dataclasses import dataclass

@dataclass
class User:
    id: str
    phone_number: int
    
def default(obj):
    
    if isinstance(obj, User):
        phone_number_str = str(obj.phone_number)
        return {
            'id': obj.id,
            'phone_number': f'{phone_number_str[:3]}xxxx{phone_number_str[-4:]}'
        }
    
    raise TypeError
    
demo_json6 = {
    'user1': User(id=str(uuid.uuid4()), phone_number=13966666666)
}
    
orjson.dumps(demo_json6,
             option=orjson.OPT_PASSTHROUGH_DATACLASS,
             default=default)

b'{"user1":{"id":"d57326d0-e8f5-490f-9271-74b617e25a3e","phone_number":"139xxxx6666"}}'

In [23]:
def default(obj):
    
    if isinstance(obj, datetime):
        return obj.strftime('%Y年%m月%d日')
    
    raise TypeError

demo_json7 = {
    'now': datetime.now()
}

orjson.dumps(demo_json7,
             option=orjson.OPT_PASSTHROUGH_DATETIME,
             default=default).decode()

'{"now":"2022年06月05日"}'